In [ ]:
# This script finds all duplicate resources and their parent (item) or children (asset) in Academic Commons (AC).
#
# First created: 2022-11-10
# Finalized: 2022-11-22
# 2nd Edition: 2022-12-22
#
# Main Processes:
# 1) Import (a) the complete data exported from AC, and (b) a list of current duplicates in AC.
# 2) Select items from (b) that are marked as duplicates ('Yes dupe').
# 3) Look up bulk AC data for dupllicates' child assets.
# 4) Output as 2 CSV, one for Hyacinth, the other one for DataCite. See descriptions below.
#
# 2nd Edition:
# To merge stats of duplicate items and assets on Hyacinth, mapping duplicate child assets to its stay-published equivalent.
# It should be the first child asset that is:
# (a) Published on AC ('Publish Target 1 > String Key' = academic_commons), and
# (b) 'Asset Data > Original Filename' is not 'mets.xml', which should never be published.

import pandas as pd

In [ ]:
# **Import the AC exported full dataset**

df= pd.read_csv('ac_export_data.csv', dtype='string')

# df.head()      # Sample data
# print(df.columns.tolist())     # Total 8092 columns

In [ ]:
# **Import the PID list from "Duplicates in AC" to compare with the main list.**

currentACDupe = pd.read_csv('currentPIDList.csv')

# currentACPID_list.head()       # Sample data.

In [ ]:
# Extract only the relevent columns from the full AC data to speed up process.

trimmedACData = df[['PID', '_doi', 'Digital Object Type > String Key', 'Title 1 > Sort Portion', 'Parent Digital Object 1 > PID', 'Asset Data > Original Filename', 'Asset Data > Checksum', 'Publish Target 1 > String Key']]
trimmedACData.columns = ['PID', 'DOI', 'Object Type', 'Title', 'Parent PID', 'Filename', 'Checksum', 'Published']
trimmedACData = trimmedACData.drop([0])     # Remove first row of element keys.

trimmedACData.head(3)        # Sample data

In [ ]:
# Select rows that are marked as duplicates from the "Duplicate in AC" CSV.

currentACDupe = currentACDupe[currentACDupe['YES dupe'] == True].reset_index(drop=True)

# Create a list of duplicate PIDs.

currentDupePID = currentACDupe['delete--PID'].tolist()

currentACDupe.head(3)

In [ ]:
# If any resource in the current dupe list is a parent item, find all of its children (other assets under that parent item).
# These new found children will join the current dupe list to be worked on.
# New list shows: Child index - Child PID - Child Resource Type - Child Title - Parent PID
# A parent PID can be repeated on multiple rows if having more than one child

# isin() checks each value of the currentDupePID[] if exist in the parent object column in bulk AC data.
childofDupeParent = trimmedACData[trimmedACData['Parent PID'].isin(currentDupePID)]
#childofDupeParent.columns = ['PID', 'DOI', 'Object Type', 'Title', 'Parent PID', 'Filename', 'Published']

childofDupeParent

In [ ]:
# Mapping new parent items' PID and DOI information for the child assets found in last step.
# The new column 'DOI to Map to' will be the DOIs that the duplicates should be redirected to.
# Information of new DOIs comes from the current AC duplicate list.

parentNewDOI = currentACDupe[['delete--PID', 'keep--PID', 'keep--DOI']]

childofDupeParent = childofDupeParent.merge(parentNewDOI, how='left', left_on='Parent PID', right_on='delete--PID', sort=True)
childofDupeParent = childofDupeParent.drop(columns=['delete--PID'])
childofDupeParent = childofDupeParent.rename(columns={'keep--PID': 'Parent PID to Keep', 'keep--DOI': 'DOI to Map to'})

childofDupeParent

In [ ]:
# Merge duplicate parents' children data to currentACDupe
# Parent and chilren are listed together
# Resulting list contains:
# DOI - PID - Object Type - Title - New PID - New DOI (to map to)

currentACDupSubset = currentACDupe[['delete--DOI', 'delete--PID', 'OR Digital Object Type > String Key', 'OR Title 1 > Sort Portion', 'keep--PID', 'keep--DOI']]
currentACDupSubset.columns = ['DOI', 'PID', 'Object Type', 'Title', 'Parent PID to Keep', 'DOI to Map to']
# currentACDupSubset

allItemsMapped = pd.concat([currentACDupSubset, childofDupeParent], ignore_index=True)

allItemsMapped = allItemsMapped.drop(columns=['Parent PID'])
allItemsMapped = allItemsMapped.sort_values(by='PID')
allItemsMapped.reset_index(inplace=True, drop=True)
allItemsMapped = allItemsMapped.replace(r'\r', r'', regex=True)   # Remove irrelevant new lines from values

# Reformat 'keep--DOI' as actual URL

allItemsMapped['DOI to Map to'] = allItemsMapped['DOI to Map to'].str.replace(r'doi:', 'https://academiccommons.columbia.edu/doi/', regex=True)

#allItemsMapped.to_csv('all_dupe_and_related.csv')
allItemsMapped.head(3)

In [ ]:
# 2nd Edition new function: child asset level mapping
# Mapping duplicate child assets to its stay-published equivalent for Hyacinth to merge the stats before removal.

# Create a list of parent items to be kept.
keepParentPID = allItemsMapped['Parent PID to Keep'].unique()
keepParentPIDlist = keepParentPID.tolist()

# Collect all child assets of each keeping parent.
childofKeepParent = trimmedACData[trimmedACData['Parent PID'].isin(keepParentPIDlist)]

# Select duplicate and keeping assets that are published on Academic Commons, and are not "mets.xml"
selectedDupeChild = allItemsMapped[allItemsMapped['Object Type'] == 'asset']
selectedDupeChild = selectedDupeChild[selectedDupeChild['Published'] == 'academic_commons']
selectedDupeChild = selectedDupeChild[selectedDupeChild['Filename'] != 'mets.xml']

childofKeepParent = childofKeepParent[childofKeepParent['Published'] == 'academic_commons']
childofKeepParent = childofKeepParent[childofKeepParent['Filename'] != 'mets.xml']

# Mapping duplicate child assets to its stay-published equivalent by Parent PID
allChildMapped = selectedDupeChild.merge(childofKeepParent, how='left', left_on=['Parent PID to Keep'], right_on=['Parent PID'], sort=True)

# Cleaning up duplicated rows resulted from non-1-to-1 assets mapping.
# Cases include 1 dupe to multiple keeping assets, multiple dupes to 1 keeping asset, or multiple dupes to multiple keeping assets.
subsetDupes = allChildMapped.duplicated(['Parent PID'], keep=False)

subsetDupesCleanUp = allChildMapped[subsetDupes]

# Keeping dupes that match either the filename or the checksum of a keeping asset.
# Hyacinth's stats only counts the most downloaded assets, the rest don't matter.
# A loop hole here is in a multiple to multiple matching, the duplicate and keeping items might share no matching files.
# Example: The main dupe asset misses a copyright page (different checksum), and the filename different from the keeping asset.
# Will need another parameter (e.g. download count) to do such a matching.

subsetDupesCleanUp = subsetDupesCleanUp.loc[((allChildMapped['Filename_x'] == allChildMapped['Filename_y']) | (allChildMapped['Checksum_x'] == allChildMapped['Checksum_y']))]

# After cleaning up duplicate matches, rejoin list of parents that have 1 child asset only.
DupesCleaned = pd.concat([allChildMapped[~subsetDupes], subsetDupesCleanUp])

# Further remove duplicates that have identical assets mapped to the same keeping asset (which still left multiple rows from last step).
DupesCleaned.drop_duplicates(['Parent PID'], keep='first', inplace=True, ignore_index=True)

# Prepare output by resetting index, selecting and renaming columns.
DupesCleaned.sort_values(by=['Parent PID', 'PID_x'], inplace=True)
DupesCleaned.reset_index(inplace=True, drop=True)
DupesCleaned = DupesCleaned[['Parent PID', 'PID_x', 'Filename_x', 'PID_y', 'Filename_y']]
DupesCleaned.columns = ['Parent PID to Keep', 'Dupe Asset PID', 'Dupe Asset Filename', 'Keeping Asset PID', 'Keeping Asset Filename']

DupesCleaned.to_csv('Hyacinth_Child_Level_Mapping.csv')

DupesCleaned.head(3)

In [ ]:
# Sort and export the list to work on Hyacinth
# The first column is PID

hyacinthList = allItemsMapped[['PID', 'DOI', 'Title', 'Object Type', 'Parent PID to Keep', 'DOI to Map to']]

# Export as CSV file
hyacinthList.to_csv('all_dupe_Hyacinth.csv')

hyacinthList

In [ ]:
# Export the list to work on DataCite.
# Those without a DOI will be removed.

final_for_DataCite = allItemsMapped.dropna(subset=['DOI'])

# Swapping the columns of PID and DOI
final_for_DataCite = final_for_DataCite.reindex(columns=['DOI', 'PID', 'Object Type', 'DOI to Map to'])
final_for_DataCite.reset_index(inplace=True, drop=True)

# Export as CSV file
final_for_DataCite.to_csv('all_dupe_Datacite.csv')
final_for_DataCite